# Derrière le pipeline (PyTorch)

Installez la bibliothèque 🤗 *Transformers* pour exécuter ce *notebook*.

In [2]:
%pip install "transformers[sentencepiece]"
%pip install torch

You should consider upgrading via the '/Users/ashnaahmad/Documents/GitHub/course/.env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/ashnaahmad/Documents/GitHub/course/.env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="tblard/tf-allocine")
classifier(
    ["J'ai attendu un cours d'HuggingFace toute ma vie.",
     "Je déteste tellement ça !"]
)

Comme nous l'avons vu dans le [chapitre 1](/course/fr/chapter1), ce pipeline regroupe trois étapes : le prétraitement, le passage des entrées dans le modèle et le post-traitement.

Pour ce faire, nous utilisons un *tokenizer*, qui sera responsable de :
- **diviser l'entrée en mots, sous-mots, ou symboles** (comme la ponctuation) qui sont appelés *tokens*,
- **associer chaque *token* à un nombre entier**,
- **ajouter des entrées supplémentaires** qui peuvent être utiles au modèle.

Tout ce prétraitement doit être effectué exactement de la même manière que celui appliqué lors du pré-entraînement du modèle. **Nous devons donc d'abord télécharger ces informations depuis le [*Hub*](https://huggingface.co/models). Pour ce faire, nous utilisons la classe `AutoTokenizer` et sa méthode `from_pretrained()`**. En utilisant le nom du *checkpoint* de notre modèle, elle va automatiquement **récupérer les données associées au *tokenizer* du modèle** et les mettre en cache (afin qu'elles ne soient téléchargées que la première fois que vous exécutez le code ci-dessous).



In [3]:
from transformers import AutoTokenizer

checkpoint = "tblard/tf-allocine" # pre-traitement, effectué du même manière que le pre-entraînement du modèle
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# **Checkpoints** : ce sont les **poids qui seront chargés** dans une architecture donnée.

/Users/ashnaahmad/Documents/GitHub/course/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Une fois que nous avons le *tokenizer* nous pouvons lui passer directement nos phrases et obtenir un dictionnaire prêt à être donné à notre modèle ! La seule chose qui reste à faire est de convertir en tenseurs la liste des identifiants d'entrée.

Vous pouvez utiliser 🤗 *Transformers* sans avoir à vous soucier du *framework* utilisé comme *backend*. Il peut s'agir de PyTorch, de TensorFlow ou de Flax pour certains modèles. Cependant, **les *transformers* n'acceptent que les *tenseurs* en entrée.** Si c'est la première fois que vous entendez parler de tenseurs, vous pouvez les considérer comme des tableaux NumPy. Un tableau NumPy peut être un scalaire (0D), un vecteur (1D), une matrice (2D), ou avoir davantage de dimensions. Les tenseurs des autres *frameworks* d'apprentissage machine se comportent de manière similaire et sont généralement aussi simples à instancier que les tableaux NumPy.

Pour spécifier le type de tenseurs que nous voulons récupérer (PyTorch, TensorFlow, ou simplement NumPy), nous utilisons l'argument `return_tensors` :

In [4]:
raw_inputs = [
    "J'ai attendu un cours d'HuggingFace toute ma vie.",
    "Je déteste tellement ça !",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    5,   121,    11,    73,  4903,    23,   307,    18,    11, 10312,
         20841,   449,  5702,   194,   155,   157,     9,     6],
        [    5,   100,  8645,  1217,   136,    83,     6,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


La sortie elle-même est un dictionnaire contenant deux clés : `input_ids` et `attention_mask`. `input_ids` contient deux lignes d'entiers (une pour chaque phrase) qui sont les identifiants uniques des *tokens* dans chaque phrase. Nous expliquerons ce qu'est l'`attention_mask` plus tard dans ce chapitre. 

Nous pouvons **télécharger notre modèle pré-entraîné** de la même manière que nous l'avons fait avec notre *tokenizer*. 🤗 *Transformers* fournit une classe `AutoModel` qui possède également une méthode `from_pretrained()` :

Dans cet extrait de code, **nous avons téléchargé le même *checkpoint*** que nous avons utilisé dans notre pipeline auparavant (il devrait en fait avoir déjà été mis en cache) et instancié un modèle avec lui.

# **Checkpoints** : ce sont les **poids qui seront chargés** dans une architecture donnée.

Cette architecture ne contient que **le module de *transformer* de base** : **étant donné certaines entrées, il produit ce que nous appellerons des *états cachés***, également connus sous le nom de *caractéristiques*. 
Pour chaque entrée du modèle, nous récupérons un vecteur en grande dimension représentant la **compréhension contextuelle de cette entrée par le *transformer***.

Si cela ne fait pas sens, ne vous inquiétez pas. Nous expliquons tout plus tard.

Bien que ces états cachés puissent être utiles en eux-mêmes, ils sont généralement **les entrées d'une autre partie du modèle, connue sous le nom de *tête***. Dans le [chapitre 1](/course/fr/chapter1), les différentes tâches auraient pu être **réalisées avec la même architecture mais en ayant chacune d'elles une tête différente.**

(repeat cell for a reminder of what's going on in the cell below)

Tout ce prétraitement doit être effectué exactement de la même manière que celui appliqué lors du pré-entraînement du modèle. **Nous devons donc d'abord télécharger ces informations depuis le [*Hub*](https://huggingface.co/models). Pour ce faire, nous utilisons la classe `AutoTokenizer` et sa méthode `from_pretrained()`**. En utilisant le nom du *checkpoint* de notre modèle, elle va automatiquement **récupérer les données associées au *tokenizer* du modèle** et les mettre en cache (afin qu'elles ne soient téléchargées que la première fois que vous exécutez le code ci-dessous).


In [7]:
from transformers import AutoModel

checkpoint = "philschmid/pt-tblard-tf-allocine"
model = AutoModel.from_pretrained(checkpoint, from_tf=True)

OSError: philschmid/pt-tblard-tf-allocine does not appear to have a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

Dans cet extrait de code (ci-dessus), nous avons téléchargé le même *checkpoint* que nous avons utilisé dans notre pipeline auparavant (il devrait en fait avoir déjà été mis en cache) et instancié un modèle avec lui.

Cette architecture ne contient que le module de *transformer* de base : étant donné certaines entrées, il produit ce que nous appellerons des *états cachés*, également connus sous le nom de *caractéristiques*. 
Pour chaque entrée du modèle, nous récupérons un vecteur en grande dimension représentant la **compréhension contextuelle de cette entrée par le *transformer***. (ahhhhh)

Si cela ne fait pas sens, ne vous inquiétez pas. Nous expliquons tout plus tard.

**les états cachés (qui correspondent à un certain ensemble de valeurs d'entrée) - sont représentés par un vecteur et représentent la comprehension contextuelle du modèle. cette comprehension se compose normalement d'entrées qui se trouvent ailleurs dans le modèle. cet ensemble d'entrées constitue la tête du modèle. chaque tête peut correspondre à une réalisation d'une tâche différente avec l'architecture.**

Bien que ces états cachés puissent être utiles en eux-mêmes, ils sont généralement les entrées d'une autre partie du modèle, connue sous le nom de *tête*. Dans le [chapitre 1](/course/fr/chapter1), les différentes tâches auraient pu être réalisées avec la même architecture mais en ayant chacune d'elles une tête différente.

**Le vecteur produit en sortie par le *transformer* est généralement de grande dimension**. Il a généralement trois dimensions :

- **la taille du lot** : le **nombre de séquences traitées à la fois** (2 dans notre exemple),
- **la longueur de la séquence** : la longueur de la représentation numérique de la séquence (16 dans notre exemple),
- **la taille cachée** : la **dimension du vecteur** de **chaque entrée** du modèle.

On dit qu'il est de « grande dimension » en raison de la dernière valeur. La taille cachée peut être très grande (généralement 768 pour les petits modèles et pour les grands modèles cela peut atteindre 3072 voire plus).

**Nous pouvons le constater** si nous alimentons notre modèle avec les entrées que nous avons prétraitées :

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)
# prints torch.size() object
# you can access elements of the output via index or attribute/key as in a dictionary

model input -> transformer (embeddings, layers) -> hidden states (contextual understanding) -> head -> model output

Les têtes des modèles **prennent en entrée le vecteur de grande dimension des états cachés** et le projettent sur une autre dimension. Elles sont généralement composées d'une ou de quelques **couches linéaires** :


**La sortie du *transformer* est envoyée directement à la tête du modèle pour être traitée.**
Dans ce diagramme, le modèle est représenté par sa couche d’enchâssement et les couches suivantes. **La couche d’enchâssement convertit chaque identifiant d'entrée dans l'entrée tokenisée en un vecteur qui représente le *token* associé**. Les couches suivantes manipulent ces vecteurs en utilisant le mécanisme d'attention pour produire la représentation finale des phrases.

les têtes sont adaptés aux tâches - comme le premier paramètre de la fonction pipeline

tant pis les têtes sont importées

Il existe de nombreuses architectures différentes disponibles dans la bibliothèque 🤗 *Transformers*, chacune étant conçue autour de la prise en charge d'une tâche spécifique. En voici une liste non exhaustive :
- `*Model` (récupérer les états cachés)
- `*ForCausalLM`
- `*ForMaskedLM`
- `*ForMultipleChoice`
- `*ForQuestionAnswering`
- `*ForSequenceClassification`
- `*ForTokenClassification`
- et autres 🤗

Pour notre exemple, nous avons besoin d'un modèle avec une tête de classification de séquence (pour pouvoir classer les phrases comme positives ou négatives). Donc, nous n'utilisons pas réellement la classe `AutoModel` mais plutôt `AutoModelForSequenceClassification` :

In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "tblard/tf-allocine"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, from_tf=True)
outputs = model(**inputs)

Maintenant, si nous examinons la forme de nos entrées, la dimensionnalité est beaucoup plus faible. La tête du modèle prend en entrée les vecteurs de grande dimension que nous avons vus précédemment et elle produit des vecteurs contenant deux valeurs (une par étiquette) :

In [ ]:
print(outputs.logits.shape)

Comme nous n'avons que deux phrases et deux étiquettes, le résultat que nous obtenons est de forme 2 x 2

## Post-traitement de la sortie

Les valeurs que nous obtenons en sortie de notre modèle n'ont pas nécessairement de sens en elles-mêmes. Jetons-y un coup d’œil  :

In [ ]:
print(outputs.logits)

Notre modèle a prédit `[-1.5607, 1.6123]` pour la première phrase et `[ 4.1692, -3.3464]` pour la seconde. **Ce ne sont pas des probabilités mais des *logits*, les scores bruts, non normalisés, produits par la dernière couche du modèle**. Pour être convertis en probabilités, **ils doivent passer par une couche [SoftMax](https://fr.wikipedia.org/wiki/Fonction_softmax)** (tous les modèles de la bibliothèque 🤗 *Transformers* sortent les logits car **la fonction de perte de l'entraînement fusionne généralement la dernière fonction d'activation, comme la SoftMax, avec la fonction de perte réelle, comme l'entropie croisée**) :

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

Maintenant nous pouvons voir que le modèle a prédit `[0.0402, 0.9598]` pour la première phrase et `[0.9995, 0.0005]` pour la seconde. Ce sont des scores de probabilité reconnaissables.

Pour obtenir les étiquettes correspondant à chaque position, **nous pouvons inspecter l'attribut `id2label` de la configuration du modèle** (plus de détails dans la section suivante)

In [ ]:
model.config.id2label


Nous pouvons maintenant conclure que le modèle a prédit ce qui suit :
 
- première phrase : NEGATIVE: 0.0402, POSITIVE: 0.9598
- deuxième phrase : NEGATIVE: 0.9995, POSITIVE: 0.0005

Nous avons reproduit avec succès les trois étapes du pipeline : prétraitement avec les *tokenizers*, passage des entrées dans le modèle et post-traitement ! Prenons maintenant le temps de nous plonger plus profondément dans chacune de ces étapes.